In [137]:
import pickle
import pandas as pd


def pretty_print(df1: pd.DataFrame, 
                 df2: pd.DataFrame, 
                 column: str, 
                 mode: str = "of"):
    if mode == "of":
        col1 = "MAX_PROFIT"
        col2 = "MAX_PARCELS"
        empty = " "
      
    elif mode == "10":
        col1 = "10"
        col2 = "no_10"
        empty = ""

    print(column)
    print(f" {col1:<10}: {empty}{df1[column].mean(): .2f} +/- {round(df1[column].std(), 2)}", "\n",
          f"{col2:<10}: {df2[column].mean(): .2f} +/- {round(df2[column].std(),2)}", "\n",)



def print_stats(df1: pd.DataFrame, 
                df2: pd.DataFrame,
                mode: str = "of"):
    if mode == "of":
        col1 = "MAX_PROFIT"
        col2 = "MAX_PARCELS"
      
    elif mode == "10":
        col1 = "10"
        col2 = "no_10"

    for col in ["profit", "parcels", "runtime"]:
      pretty_print(df1, df2, col, mode)

    print(f"--- DIFF ({col1} - {col2}) ---")
    print("Mean diff in profit", 
          round(df1["profit"].mean() - df2["profit"].mean(),2))
    print("Mean diff in parcels", 
          round(df1["parcels"].mean() - df2["parcels"].mean(),2))
    print("Mean diff in runtime", 
          round(df1["runtime"].mean() - df2["runtime"].mean(), 2))


def analyze_results(path: str, 
                    mode: str = "of"):
    with open(path, "rb") as f:
        results, seeds = pickle.load(f)

    if mode == "of":
        df1 = pd.DataFrame.from_dict(results["MAX_PROFIT"]).transpose()
        df2 = pd.DataFrame.from_dict(results["MAX_PARCELS"]).transpose()

    elif mode == "10":
        df1 = pd.DataFrame.from_dict(results["10"]).transpose()
        df2 = pd.DataFrame.from_dict(results["no_10"]).transpose()

    print_stats(df1, df2, mode)

    return df1, df2, seeds

In [132]:
paths = [
    "../num_analysis/results_of.pkl",
    "../num_analysis/results_10_bin.pkl",
    "../num_analysis/results_10_bin_2.pkl",
    "../num_analysis/results_10_uni_4.pkl"
]

### Analysis for the Objective Function

- MAX_PROFIT maximiert gleichzeitig auch die Anzahl an Paketen
- Keine großen Unterschiede in der Rechenzeit; MAX_PARCELS leicht schneller
- MAX_PROFIT hat offensichtlich viel höheren Profit

In [138]:
max_profit, max_parcels, seeds_of = analyze_results(paths[0], 
                                                    mode="of")

profit
 MAX_PROFIT:   78.62 +/- 10.13 
 MAX_PARCELS:  16.70 +/- 11.69 

parcels
 MAX_PROFIT:   23.80 +/- 2.96 
 MAX_PARCELS:  23.80 +/- 2.96 

runtime
 MAX_PROFIT:   17.65 +/- 4.71 
 MAX_PARCELS:  14.19 +/- 4.95 

--- DIFF (MAX_PROFIT - MAX_PARCELS) ---
Mean diff in profit 61.92
Mean diff in parcels 0.0
Mean diff in runtime 3.47


### Analysis for Restriction 10 and l_s

- Rechenzeit von Modell ohne Restriktion 10 deutlich höher
- Je kleiner die Kapazitäten, desto öfter treten fehlerhafte Instanzen auf
- Rechenzeit ohne Restriktion 10 steigt relativ stark für niedrigere Kapazitäten; 
mit 10 bleibt relativ konstant über verschiedene Kapazitäten

In [139]:
ten_bin, no_ten_bin, seeds_10_bin = analyze_results(paths[1], 
                                                    mode="10")

profit
 10        :  75.34 +/- 10.03 
 no_10     :  75.42 +/- 10.02 

parcels
 10        :  22.90 +/- 3.03 
 no_10     :  22.92 +/- 3.04 

runtime
 10        :  16.71 +/- 4.5 
 no_10     :  54.29 +/- 22.27 

--- DIFF (10 - no_10) ---
Mean diff in profit -0.08
Mean diff in parcels -0.02
Mean diff in runtime -37.58


In [140]:
ten_bin_2, no_ten_bin_2, seeds_10_bin_2 = analyze_results(paths[2], 
                                                          mode="10")

profit
 10        :  74.12 +/- 10.26 
 no_10     :  74.40 +/- 10.38 

parcels
 10        :  22.62 +/- 3.02 
 no_10     :  22.74 +/- 3.08 

runtime
 10        :  17.58 +/- 5.49 
 no_10     :  97.36 +/- 159.52 

--- DIFF (10 - no_10) ---
Mean diff in profit -0.28
Mean diff in parcels -0.12
Mean diff in runtime -79.79


In [149]:
ten_uni_4, no_ten_uni_4, seeds_10_uni_4 = analyze_results(paths[3], 
                                                          mode="10")

profit
 10        :  71.82 +/- 10.49 
 no_10     :  72.52 +/- 10.42 

parcels
 10        :  22.10 +/- 3.01 
 no_10     :  22.30 +/- 3.0 

runtime
 10        :  15.97 +/- 3.69 
 no_10     :  86.23 +/- 83.66 

--- DIFF (10 - no_10) ---
Mean diff in profit -0.7
Mean diff in parcels -0.2
Mean diff in runtime -70.26


In [151]:
no_ten_bin_2["runtime"].median(), ten_bin_2["runtime"].median(), no_ten_uni_4["runtime"].median(), ten_uni_4["runtime"].median(), no_ten_bin["runtime"].median(), ten_bin["runtime"].median()

(47.650999999999996,
 16.514499999999998,
 57.2525,
 15.603,
 48.801,
 16.435000000000002)